# 0 - Imports

In [1]:
import sys
import joblib

# Retornar para a pasta 'src' para poder importar os módulos criados em 'data/', 'utils/', etc
sys.path.insert(0, '../src/')


# import yaml consig file
from utils.utils import load_config_file

# Data load step
from data.data_load import DataLoad 

# Data validation step
from data.data_validation import DataValidation 

# Data transformation step
from data.data_transformation import DataTransformation 

# Data preprocessing step
from data.data_preprocessing import DataPreprocessing

# pipeline
from sklearn.pipeline import Pipeline

# imputers and discretizers
from feature_engine.imputation import MeanMedianImputer
from feature_engine.discretisation import EqualFrequencyDiscretiser, EqualWidthDiscretiser

# scaler
from sklearn.preprocessing import StandardScaler

# sklearn wrapper
from feature_engine.wrappers import SklearnTransformerWrapper

# import yaml consig file
from utils.utils import load_config_file

# Train model step
from train.train import ModelTraining

# Model validation step
from evaluation.classifier_evaluation import ModelEvaluating


from sklearn.linear_model import LogisticRegression

In [2]:
# Loading yaml file
yaml_file = load_config_file()

# 1 - Data loading

In [3]:
# Instantiating 'data_loader' object

data_loader = DataLoad()

# loading csv file as dataframe from yaml config file using 'load_data()' method 

df = data_loader.load_data('train_dataset_name')

df.head()

2024-03-24 19:09:25 [info     ] Staring data loading with: train_dataset_name
2024-03-24 19:09:26 [info     ] Data loaded successfully!


,target,TaxaDeUtilizacaoDeLinhasNaoGarantidas,Idade,NumeroDeVezes30-59DiasAtrasoNaoPior,TaxaDeEndividamento,RendaMensal,NumeroDeLinhasDeCreditoEEmprestimosAbertos,NumeroDeVezes90DiasAtraso,NumeroDeEmprestimosOuLinhasImobiliarias,NumeroDeVezes60-89DiasAtrasoNaoPior,NumeroDeDependentes
0,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2
1,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1
2,0,0.65818,38,1,0.085113,3042.0,2,1,0,0,0
3,0,0.23381,30,0,0.03605,3300.0,5,0,0,0,0
4,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0


# 2 - Data checking

In [4]:
# Instantiating 'data_validator' object

data_validator = DataValidation()


# run data validation
data_validator.run(df)

2024-03-24 19:09:27 [info     ] Starting data shape checking
2024-03-24 19:09:27 [info     ] Data shape validation started!
2024-03-24 19:09:27 [info     ] Starting column types validation
2024-03-24 19:09:27 [info     ] Column validation passed!
2024-03-24 19:09:27 [info     ] Validation step concluded!


True

# 3 - Data transformation (split)

In [5]:
# Instantiating 'data_transformer' object

data_transformer = DataTransformation(df)


# getting X_train, X_val, y_train, y_val splits
X_train, X_val, y_train, y_val = data_transformer.train_test_data_split()

2024-03-24 19:09:28 [info     ] Target 'target' variable found in the dataframe. Performing X, y split and train/val split
2024-03-24 19:09:28 [info     ] Data split performed successfully!


# 4 - Experimentations (MLFlow)

In [6]:
import mlflow
from mlflow.tracking import MlflowClient

In [7]:
# Definir o servidor do MLFlow:
# fazer a ligação entre os códigos executados nesse notebook com a UI do MLFlow
mlflow.set_tracking_uri("http://127.0.0.1:5000/")

#mlflow.set_tracking_uri("sqlite:///mlruns.db")

# Criar um novo experimento:
mlflow.set_experiment(yaml_file.get('experiment_name')) # definir o nome

<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1711251631903, experiment_id='1', last_update_time=1711251631903, lifecycle_stage='active', name='prob_loan_exp', tags={}>

## 4.1 - Hyperparameter Fine Tuning

In [8]:
# Experimento de fine tuning de hiperparâmetros c/ Hyperopt

In [9]:
# Importar módulo que salva o experimento apenas de atingir um threshold predefinido
from mlflow.models import MetricThreshold, infer_signature

# Importar classificador dummy para comparação
from sklearn.dummy import DummyClassifier

# Importar métodos da biblioteca hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK

In [10]:
# 1.0 Preprocessar os dados: =============================================
pipeline =  Pipeline(
                [
                 ('imputer', MeanMedianImputer(variables=yaml_file['variables_imputer'])),
                 ('discretizer', EqualFrequencyDiscretiser(variables=yaml_file['variables_discretiser'])),                  
                 ('scaler', SklearnTransformerWrapper(StandardScaler()))
                                                                          ]
                  )

In [11]:

# definição de função objetiva p/ Hyperopt:

def objective_func(params):
    
    # Criar um experimento base (execução/run)

    with mlflow.start_run(run_name='with_discretizer_hyperopt'):

        mlflow.set_tag('model_name', 'lr_discretizer_hyperopt')

        mlflow.log_params(params)


        # Instantiating 'data_preprocessor' object from DataPreprocessing class
        data_preprocessor = DataPreprocessing(pipeline)

        # Fitting pipeline
        data_preprocessor.fit_pipeline(X_train)


        # Transforming data

        X_train_processed = data_preprocessor.transform_pipeline(X_train) # X_train

        X_val_processed = data_preprocessor.transform_pipeline(X_val) # X_val

        joblib.dump(data_preprocessor, '../models/preprocessor.joblib')

        # ------------------------------------------------------------------------------


        # 1.1 logar um artefato (proprocessor)
        mlflow.log_artifact('../models/preprocessor.joblib')

        # 1.2 logar os parâmetros do proprocessador
        mlflow.log_params(params={'imputer':pipeline['imputer'],
                                  'discretizer':pipeline['discretizer'],
                                  'scaler':pipeline['scaler']})
    


        # 2.0 Treinamento do modelo e Cross validation: =============================================

        model_instance = LogisticRegression(**params)

        # Instantiating 'model_trainer' object from DataPreprocessing class
        model_trainer = ModelTraining(X_train_processed, y_train)

        # Fitting model
        fitted_model = model_trainer.fit(model_instance)

        # Instantiating 'model_evaluator' object from DataPreprocessing class
        model_evaluator = ModelEvaluating(fitted_model, X_train_processed, y_train)


        # Getting metric scores
        scores = model_evaluator.cross_val_evaluation()

        # ------------------------------------------------------------------------------


        # 2.1 logar o resultado da métrica do modelo
        mlflow.log_metric('CV_roc_auc', scores.mean())



        # 3.0 Treinamento do modelo =============================================

        # Fitting model
        model_instance.fit(X_train_processed, y_train)


        # 4.0 Salvar metricas do X_val
        y_val_probas = model_instance.predict_proba(X_val_processed)[:,1]
        X_val_roc_auc = model_evaluator.evaluate_predictions(y_val, y_val_probas)

        # logar o resultado da métrica do modelo
        mlflow.log_metric('Val_roc_auc', X_val_roc_auc)



        # 5.0 Logar o modelo candidato e capturar a uri para mlflow.evaluate
        model_candidate_uri = mlflow.sklearn.log_model(model_instance,
                                                   'lr_model').model_uri


        # ===============================

        # Inferir assinatura dos dados (aparece o dataset no MLFlow)
        signature = infer_signature(X_val_processed, y_val)

        # Separar os dados para avaliação do MLFlow (p/ usar no método mlflow.evaluate()):
        eval_data = X_val_processed.copy() # criando uma cópia de X_val_processed
        eval_data['label'] = y_val # criando a coluna target


        # Definindo o threshold do experimento
        thresholds = {
            yaml_file.get('metric_to_evaluate'): MetricThreshold(

                threshold=yaml_file.get('metric_threshold'), # a métrica precisa ser maior que esse valor para que o experimento seja salvo

                min_absolute_change=yaml_file.get('metric_min_absolute_change'),

                min_relative_change=yaml_file.get('metric_min_relative_change'),

                greater_is_better=yaml_file.get('greater_is_better')
            )
        }

        # Instanciando o classificador dummy
        baseline_model_instance = DummyClassifier(strategy='uniform').fit(X_train_processed, y_train)


        # Logar o modelo dummy e capturar a uri para mlflow.evaluate
        baseline_model_instance_uri = mlflow.sklearn.log_model(baseline_model_instance,
                                                               'baseline_model',
                                                               signature=signature).model_uri



        # Iniciar avaliação:

        # processo responsável por avaliar/comparar modelos no MLFlow
        mlflow.evaluate(model_candidate_uri,
                        eval_data,
                        targets='label',
                        model_type=yaml_file.get('model_type'),
                        validation_thresholds=thresholds,
                        baseline_model=baseline_model_instance_uri)



        # end run
        mlflow.end_run()

        return {'loss': -scores.mean(),
                'status': STATUS_OK}


In [12]:
# Definindo o espaço de busca de hiperparâmetros

search_space = {'warm_start': hp.choice('warm_start',[True, False]),
                'fit_intercept': hp.choice('fit_intercept',[True, False]),
                'tol': hp.uniform('tol',0.00001, 0.001),
                'C': hp.uniform('C', 0.05, 3),
                'solver': hp.choice('solver',['newton-cg', 'lbfgs','liblinear']),
                'max_iter': hp.choice('max_iter',range(100, 1000)),
                'multi_class': 'auto',
                'class_weight': hp.choice('class_weight',[None, 'balanced']),}

In [13]:
best_result = fmin(fn=objective_func,
                   space=search_space,
                   algo=tpe.suggest,
                   max_evals=yaml_file.get('max_evals'))

  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]

2024-03-24 16:38:30 [info     ] Pipeline started...
2024-03-24 16:38:30 [info     ] Pipeline fitting started...
2024-03-24 16:38:30 [info     ] Pipeline fitting finished!
2024-03-24 16:38:30 [info     ] Data transformation with fitted pipeline started...
2024-03-24 16:38:30 [info     ] Data transformation with fitted pipeline finished!
2024-03-24 16:38:30 [info     ] Data transformation with fitted pipeline started...
2024-03-24 16:38:30 [info     ] Data transformation with fitted pipeline finished!
/home/guhduarte/Repos/MLFlow_Loan/project/models/Logistic_Regression.joblib
2024-03-24 16:38:31 [info     ] Cross validation started...
2024-03-24 16:38:32 [info     ] Fitted model: LogisticRegression(C=1.8473737683615823, max_iter=330, solver='newton-cg',
                   tol=0.0007436467624349342, warm_start=True)
2024-03-24 16:38:32 [info     ] Scoring metric: roc_auc
2024-03-24 16:38:32 [info     ] Cross validation finished successfully!
2024-03-24 16:38:32 [info     ] Model validatio

/home/guhduarte/.conda/envs/MLFlow_Loan/lib/python3.12/site-packages/mlflow/types/utils.py:393: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(

/home/guhduarte/.conda/envs/MLFlow_Loan/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install

2024-03-24 16:38:40 [info     ] Pipeline started...  
2024-03-24 16:38:40 [info     ] Pipeline fitting started...
2024-03-24 16:38:41 [info     ] Pipeline fitting finished!
2024-03-24 16:38:41 [info     ] Data transformation with fitted pipeline started...
2024-03-24 16:38:41 [info     ] Data transformation with fitted pipeline finished!
2024-03-24 16:38:41 [info     ] Data transformation with fitted pipeline started...
2024-03-24 16:38:41 [info     ] Data transformation with fitted pipeline finished!
/home/guhduarte/Repos/MLFlow_Loan/project/models/Logistic_Regression.joblib       
2024-03-24 16:38:41 [info     ] Cross validation started...
2024-03-24 16:38:42 [info     ] Fitted model: LogisticRegression(C=2.5454619196573045, class_weight='balanced',
                   fit_intercept=False, max_iter=267, solver='liblinear',
                   tol=0.0005823661921277798)
2024-03-24 16:38:42 [info     ] Scoring metric: roc_auc
2024-03-24 16:38:42 [info     ] Cross validation finished succ

/home/guhduarte/.conda/envs/MLFlow_Loan/lib/python3.12/site-packages/mlflow/types/utils.py:393: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(

/home/guhduarte/.conda/envs/MLFlow_Loan/lib/python3.12/site-packages/mlflow/data/digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  string_columns = trimmed_df.colum

...

2024-03-24 16:54:03 [info     ] Pipeline started...   
2024-03-24 16:54:03 [info     ] Pipeline fitting started...
2024-03-24 16:54:03 [info     ] Pipeline fitting finished!
2024-03-24 16:54:03 [info     ] Data transformation with fitted pipeline started...
2024-03-24 16:54:03 [info     ] Data transformation with fitted pipeline finished!
2024-03-24 16:54:03 [info     ] Data transformation with fitted pipeline started...
2024-03-24 16:54:03 [info     ] Data transformation with fitted pipeline finished!
/home/guhduarte/Repos/MLFlow_Loan/project/models/Logistic_Regression.joblib        
2024-03-24 16:54:03 [info     ] Cross validation started...
2024-03-24 16:54:05 [info     ] Fitted model: LogisticRegression(C=2.749983367402132, fit_intercept=False, max_iter=777,
                   tol=0.0007772899586289291)
2024-03-24 16:54:05 [info     ] Scoring metric: roc_auc
2024-03-24 16:54:05 [info     ] Cross validation finished successfully!
2024-03-24 16:54:05 [info     ] Model validation star

/home/guhduarte/.conda/envs/MLFlow_Loan/lib/python3.12/site-packages/mlflow/types/utils.py:393: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(

/home/guhduarte/.conda/envs/MLFlow_Loan/lib/python3.12/site-packages/mlflow/data/digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  string_columns = trimmed_df.colum

2024-03-24 16:54:13 [info     ] Pipeline started...   
2024-03-24 16:54:13 [info     ] Pipeline fitting started...
2024-03-24 16:54:13 [info     ] Pipeline fitting finished!
2024-03-24 16:54:13 [info     ] Data transformation with fitted pipeline started...
2024-03-24 16:54:13 [info     ] Data transformation with fitted pipeline finished!
2024-03-24 16:54:13 [info     ] Data transformation with fitted pipeline started...
2024-03-24 16:54:13 [info     ] Data transformation with fitted pipeline finished!
/home/guhduarte/Repos/MLFlow_Loan/project/models/Logistic_Regression.joblib        
2024-03-24 16:54:14 [info     ] Cross validation started...
2024-03-24 16:54:15 [info     ] Fitted model: LogisticRegression(C=2.4408336711102145, fit_intercept=False, max_iter=630,
                   tol=0.0006241771362100975)
2024-03-24 16:54:15 [info     ] Scoring metric: roc_auc
2024-03-24 16:54:15 [info     ] Cross validation finished successfully!
2024-03-24 16:54:15 [info     ] Model validation sta

/home/guhduarte/.conda/envs/MLFlow_Loan/lib/python3.12/site-packages/mlflow/types/utils.py:393: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(

/home/guhduarte/.conda/envs/MLFlow_Loan/lib/python3.12/site-packages/mlflow/data/digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  string_columns = trimmed_df.colum

100%|██████████| 100/100 [15:53<00:00,  9.53s/trial, best loss: -0.8501255215381308]


# Rascunhos

In [21]:
# Obtendo o id do experimento

experiment_name = "prob_loan"
current_experiment=dict(mlflow.get_experiment_by_name(experiment_name))
experiment_id=current_experiment['experiment_id']
current_experiment

{'artifact_location': 'mlflow-artifacts:/1',
 'creation_time': 1711210130178,
 'experiment_id': '1',
 'last_update_time': 1711212335290,
 'lifecycle_stage': 'deleted',
 'name': 'prob_loan',
 'tags': {}}

In [ ]:
# Deletar experimento

import mlflow
mlflow.delete_experiment(experiment_id=1)

In [19]:
from mlflow.tracking import MlflowClient
client = MlflowClient()
all_experiments = client.search_experiments()

all_experiments

[<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1711251631903, experiment_id='1', last_update_time=1711251631903, lifecycle_stage='active', name='prob_loan_exp', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1711250708036, experiment_id='0', last_update_time=1711250708036, lifecycle_stage='active', name='Default', tags={}>]

In [14]:
import mlflow
logged_model = 'runs:/cab6bf40fbd04598b853dcdfc3a70580/lr_model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

In [15]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: lr_model
  flavor: mlflow.sklearn
  run_id: cab6bf40fbd04598b853dcdfc3a70580

In [23]:
# Obter tabela do MLFlow UI como Dataframe

runs = mlflow.search_runs('1')

runs

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.Val_roc_auc,metrics.CV_roc_auc,metrics.true_positives,metrics.log_loss,...,params.solver,params.max_iter,params.warm_start,tags.mlflow.runName,tags.mlflow.log-model.history,tags.mlflow.user,tags.mlflow.datasets,tags.model_name,tags.mlflow.source.name,tags.mlflow.source.type
0,11e3062b60574348ad33716682e13ae2,1,FINISHED,mlflow-artifacts:/1/11e3062b60574348ad33716682...,2024-03-24 19:54:13.664000+00:00,2024-03-24 19:54:23.257000+00:00,0.853043,0.850116,2346.0,0.637780,...,lbfgs,630,False,with_discretizer_hyperopt,"[{""run_id"": ""11e3062b60574348ad33716682e13ae2""...",guhduarte,"[{""name"":""2ebb0c36f4b380f19dac359abbc730ef"",""h...",lr_discretizer_hyperopt,/home/guhduarte/.conda/envs/MLFlow_Loan/lib/py...,LOCAL
1,4d0cf33873d848ae8c0ee7ee654b472e,1,FINISHED,mlflow-artifacts:/1/4d0cf33873d848ae8c0ee7ee65...,2024-03-24 19:54:03.497000+00:00,2024-03-24 19:54:13.608000+00:00,0.853221,0.850117,2345.0,0.637640,...,lbfgs,777,False,with_discretizer_hyperopt,"[{""run_id"": ""4d0cf33873d848ae8c0ee7ee654b472e""...",guhduarte,"[{""name"":""2ebb0c36f4b380f19dac359abbc730ef"",""h...",lr_discretizer_hyperopt,/home/guhduarte/.conda/envs/MLFlow_Loan/lib/py...,LOCAL
2,021043078a9a411187f01c12dd316f29,1,FINISHED,mlflow-artifacts:/1/021043078a9a411187f01c12dd...,2024-03-24 19:53:52.908000+00:00,2024-03-24 19:54:03.444000+00:00,0.851385,0.847558,2360.0,0.640328,...,lbfgs,818,False,with_discretizer_hyperopt,"[{""run_id"": ""021043078a9a411187f01c12dd316f29""...",guhduarte,"[{""name"":""2ebb0c36f4b380f19dac359abbc730ef"",""h...",lr_discretizer_hyperopt,/home/guhduarte/.conda/envs/MLFlow_Loan/lib/py...,LOCAL
3,32a05e75ae9b4cb6bfc4d1db4f9f26a5,1,FINISHED,mlflow-artifacts:/1/32a05e75ae9b4cb6bfc4d1db4f...,2024-03-24 19:53:41.737000+00:00,2024-03-24 19:53:52.858000+00:00,0.838979,0.835011,2875.0,0.675392,...,newton-cg,531,False,with_discretizer_hyperopt,"[{""run_id"": ""32a05e75ae9b4cb6bfc4d1db4f9f26a5""...",guhduarte,"[{""name"":""2ebb0c36f4b380f19dac359abbc730ef"",""h...",lr_discretizer_hyperopt,/home/guhduarte/.conda/envs/MLFlow_Loan/lib/py...,LOCAL
4,abfbce710d2949a8af6c7cb09c04e992,1,FINISHED,mlflow-artifacts:/1/abfbce710d2949a8af6c7cb09c...,2024-03-24 19:53:30.831000+00:00,2024-03-24 19:53:41.690000+00:00,0.798475,0.791855,132.0,0.204519,...,lbfgs,799,False,with_discretizer_hyperopt,"[{""run_id"": ""abfbce710d2949a8af6c7cb09c04e992""...",guhduarte,"[{""name"":""2ebb0c36f4b380f19dac359abbc730ef"",""h...",lr_discretizer_hyperopt,/home/guhduarte/.conda/envs/MLFlow_Loan/lib/py...,LOCAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,8b7d4484b2514f149797aa09c475c385,1,FINISHED,mlflow-artifacts:/1/8b7d4484b2514f149797aa09c4...,2024-03-24 19:38:50.058000+00:00,2024-03-24 19:38:58.436000+00:00,0.799371,0.792985,111.0,0.204775,...,lbfgs,637,True,with_discretizer_hyperopt,"[{""run_id"": ""8b7d4484b2514f149797aa09c475c385""...",guhduarte,"[{""name"":""2ebb0c36f4b380f19dac359abbc730ef"",""h...",lr_discretizer_hyperopt,/home/guhduarte/.conda/envs/MLFlow_Loan/lib/py...,LOCAL
98,0e8678dff9e14f70af4f697f10b8a70a,1,FINISHED,mlflow-artifacts:/1/0e8678dff9e14f70af4f697f10...,2024-03-24 19:38:40.859000+00:00,2024-03-24 19:38:50.017000+00:00,0.839000,0.834942,2875.0,0.675320,...,liblinear,267,False,with_discretizer_hyperopt,"[{""run_id"": ""0e8678dff9e14f70af4f697f10b8a70a""...",guhduarte,"[{""name"":""2ebb0c36f4b380f19dac359abbc730ef"",""h...",lr_discretizer_hyperopt,/home/guhduarte/.conda/envs/MLFlow_Loan/lib/py...,LOCAL
99,f5292869c43346babd23b7ea1f6afd00,1,FINISHED,mlflow-artifacts:/1/f5292869c43346babd23b7ea1f...,2024-03-24 19:38:30.422000+00:00,2024-03-24 19:38:40.815000+00:00,0.798674,0.792134,126.0,0.204535,...,newton-cg,330,True,with_discretizer_hyperopt,"[{""run_id"": ""f5292869c43346babd23b7ea1f6afd00""...",guhduarte,"[{""name"":""2ebb0c36f4b380f19dac359abbc730ef"",""h...",lr_discretizer_hyperopt,/home/guhduarte/.conda/envs/MLFlow_Loan/lib/py...,LOC

In [17]:
# Obter id do experimento pelo nome

from mlflow.tracking import MlflowClient

client = MlflowClient()

exp_id = client.get_experiment_by_name("prob_loan_exp").experiment_id

exp_id

'1'

In [77]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:           12Gi       1.7Gi       9.7Gi       0.0Ki       993Mi        10Gi
Swap:         4.0Gi          0B       4.0Gi
